In [ ]:
# Importação de bibliotecas padrão
import os

# Importação de bibliotecas para manipulação de dados
import pandas as pd
import numpy as np

# Importação de bibliotecas para visualização de dados
import matplotlib.pyplot as plt

# Importação de bibliotecas de aprendizado de máquina
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, learning_curve
from sklearn.metrics import accuracy_score, confusion_matrix as sk_confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report, make_scorer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from imblearn.over_sampling import SMOTE

# Importação de bibliotecas adicionais
import xgboost as xgb
from scipy.stats import randint, loguniform, uniform
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

# Importação de datasets de exemplo do scikit-learn
from sklearn.datasets import make_classification


In [ ]:
#Escolha do Modelo de ML

modelo = SVC()
#modelo =  KNeighborsClassifier()
#modelo = RandomForestClassifier()
#modelo = HistGradientBoostingClassifier()

In [ ]:
#Região de Interesse do espectro Raman

regiao = '1'
#regiao = '2'
#regiao = '3'

In [ ]:
# Definindo o diretório onde estão os arquivos de espectros Raman
diretorio_raiz = 'c:/Users/Andreia/Jupyter/TCC/elementos/R' + regiao

# Listar os diretórios (tipos de plástico)
diretorios_plasticos = [diretorio for diretorio in os.listdir(diretorio_raiz) if os.path.isdir(os.path.join(diretorio_raiz, diretorio))]

# Inicializar listas para armazenar todos os dados e rótulos
all_data = []
all_labels = []

# Para cada tipo de plástico
for idx, plastico in enumerate(diretorios_plasticos):
    diretorio_plastico = os.path.join(diretorio_raiz, plastico)
    arquivos_plastico = os.listdir(diretorio_plastico)

    # Inicializar listas para armazenar dados de cada tipo de plástico
    folder_data = []
    folder_labels = []

    # Para cada arquivo de espectro Raman
    for arquivo in arquivos_plastico:
        # Lendo o arquivo de espectro Raman
        caminho_arquivo = os.path.join(diretorio_plastico, arquivo)

        # Carregando os dados do arquivo
        espectro = np.loadtxt(caminho_arquivo, delimiter='\t', dtype='str')

        # Adicionar espectro e rótulo às listas
        folder_data.append(espectro)
        folder_labels.append(plastico)

    # Adicionar dados e rótulos deste tipo de plástico às listas totais
    all_data.extend(folder_data)
    all_labels.extend(folder_labels)

# Dividir os dados totais em treino e teste
X_train, X_test, y_train, y_test = train_test_split(all_data, all_labels, test_size=0.3, random_state=42, stratify=all_labels)

# Transformar os dados em arrays numpy
train_data = np.array(X_train)
train_labels = np.array(y_train)
test_data = np.array(X_test)
test_labels = np.array(y_test)

# Verificar a forma original dos dados
print("Forma original dos dados de treinamento:", train_data.shape)
print("Forma original dos dados de teste:", test_data.shape)

# Extrair comprimentos de onda e intensidades
train_data_reshaped = train_data[:, :, 1]
test_data_reshaped = test_data[:, :, 1]

# Verificar a forma dos dados ajustados
print("Forma ajustada dos dados de treinamento:", train_data.shape)

# Inicializar o codificador de rótulos
label_encoder = LabelEncoder()

# Ajustar o codificador aos rótulos de treino e transformá-los em valores numéricos
y_encoded_train = label_encoder.fit_transform(train_labels)

# Transformar os rótulos de teste usando o mesmo codificador
y_encoded_test = label_encoder.transform(test_labels)

# Converter os arrays numpy em DataFrames
train_data = pd.DataFrame(train_data_reshaped)
train_labels = pd.DataFrame(y_encoded_train, columns=["Label"])
test_data = pd.DataFrame(test_data_reshaped)
test_labels = pd.DataFrame(y_encoded_test, columns=["Label"])

# Verificar a forma final dos DataFrames
print("Forma final dos dados de treinamento (DataFrame):", train_data.shape)
print("Forma final dos rótulos de treinamento (DataFrame):", train_labels.shape)
print("Forma final dos dados de teste (DataFrame):", test_data.shape)
print("Forma final dos rótulos de teste (DataFrame):", test_labels.shape)

# Mostrar o valor de cada número e o significado dele
class_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print("\nMapeamento de cada número codificado para a classe original:")
for encoded_value, original_class in class_mapping.items():
    print(f"{encoded_value}: {original_class}")

In [ ]:
# Reduzir dimensionalidade dos dados de treinamento
pca = PCA(n_components=15) 
#R2 = 80, R1 = 20
train_data_pca = pca.fit_transform(train_data_reshaped)
test_data_pca = pca.transform(test_data_reshaped)

# Converter os arrays numpy em DataFrames
train_data_pca = pd.DataFrame(train_data_pca)
test_data_pca = pd.DataFrame(test_data_pca )

In [ ]:
#Para otimizar os hiperparâmetros é feito um treinamento com a técnica Cross Validation


def validacao_cruzada(modelo, x, y, oversampling=False, diretorios_plasticos=None):
    # Inicializar o StratifiedKFold
    kfold = StratifiedKFold(n_splits=3)
    acuracias_split = []
    f1_scores_split = []
    confusoes_split = []

    for idx, (idx_treino, idx_validacao) in enumerate(kfold.split(x, y)):
        x_split_treino = x.iloc[idx_treino]
        y_split_treino = y.iloc[idx_treino].values.ravel()  # Converter para array unidimensional
        x_split_validacao = x.iloc[idx_validacao]
        y_split_validacao = y.iloc[idx_validacao].values.ravel()  # Converter para array unidimensional

        if oversampling:
            sm = SMOTE(random_state=42, k_neighbors=1)
            x_split_treino, y_split_treino = sm.fit_resample(x_split_treino, y_split_treino)

        # Treinar o modelo
        modelo.fit(x_split_treino, y_split_treino)

        # Fazer previsões
        y_pred = modelo.predict(x_split_validacao)

        # Calcular a acurácia e F1-score
        acuracia_split = accuracy_score(y_split_validacao, y_pred)
        f1_split = f1_score(y_split_validacao, y_pred, average='weighted')

        # Adicionar as métricas à lista
        acuracias_split.append(acuracia_split)
        f1_scores_split.append(f1_split)

        # Imprimir a acurácia do split atual
        print(f"Acurácia do split {idx + 1}: {acuracia_split:.4f}")
        print(f"F1-score do split {idx + 1}: {f1_split:.4f}")

        # Imprimir quantidade de amostras por classe
        #print("Distribuição das classes no conjunto de treino:")
        #print(pd.Series(y_split_treino).value_counts())
        #print("Distribuição das classes no conjunto de validação:")
        #print(pd.Series(y_split_validacao).value_counts())

    return acuracias_split, f1_scores_split, confusoes_split

In [ ]:

# Definir o espaço de hiperparâmetros para o modelo base, EXEMPLO.
param_dist = {
    'estimator__C': loguniform(0.1, 10),  # Distribuição log-uniforme para C
    'estimator__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Diferentes tipos de kernel
    'estimator__gamma': loguniform(1e-4, 1e-1),  # Distribuição log-uniforme para gamma
    'estimator__degree': randint(1, 10),  # Distribuição discreta para o grau do kernel polinomial
    'estimator__coef0': uniform(-1, 1)  # Distribuição uniforme para coef0
}


# Configurar o RandomizedSearchCV com o OneVsRestClassifier
random_search = RandomizedSearchCV(
    modelo,
    param_distributions=param_dist,
    n_iter=100,
    cv=3,
    random_state=42,
    n_jobs=-1
)

# Ajustar o RandomizedSearchCV com os dados de treinamento
random_search.fit(train_data_pca, train_labels)

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros (Random Search): ", random_search.best_params_)
print("Acurácia (Random Search): ", random_search.best_score_)

In [ ]:
# Definir o espaço de hiperparâmetros para busca, EXEMPLO
param_space = {
    'n_estimators': Integer(10, 200),  # número de árvores
    'criterion': Categorical(['gini', 'entropy']),  # função de critério
    'max_depth': Integer(1, 20),  # profundidade máxima
    'min_samples_split': Integer(2, 20),  # número mínimo de amostras para dividir
    'min_samples_leaf': Integer(1, 20)  # número mínimo de amostras para folha
}

# Realizar o Bayesian Search
bayes_search = BayesSearchCV(modelo, search_spaces=param_space, n_iter=50, cv=3, random_state=42, n_jobs=-1)
bayes_search.fit(train_data_pca, train_labels)

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros (Bayesian Search): ", bayes_search.best_params_)
print("Acurácia (Bayesian Search): ", bayes_search.best_score_)

In [ ]:
#Após definidos os hiperparâmetros, treinar o modelo com base neles

# Definir o espaço de hiperparâmetros para busca EXEMPLO
best_modelo = KNeighborsClassifier(
    n_neighbors= 10,
    weights= 'uniform',
    algorithm= 'auto', #'ball_tree', 'kd_tree', 'brute',
    p= 1
)

In [ ]:
# Definir a função de validação simples
def validacao_simples(modelo, x_train, y_train, x_val, y_val, oversampling=False, diretorios_plasticos=None):
    # Se necessário, aplicar oversampling no conjunto de treino
    if oversampling:
        sm = SMOTE(random_state=42, k_neighbors=5)
        x_train, y_train = sm.fit_resample(x_train, y_train)

    # Treinar o modelo
    modelo.fit(x_train, y_train)

    # Fazer previsões no conjunto de validação
    y_pred = modelo.predict(x_val)

    # Calcular a acurácia e F1-score
    acuracia = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='weighted')

    # Imprimir a acurácia e F1-score
    print(f"Acurácia: {acuracia:.4f}")
    print(f"F1-score: {f1:.4f}")

    # Imprimir matriz de confusão e relatório de classificação
    print("Matriz de Confusão:")
    cm = sk_confusion_matrix(y_val, y_pred)
    print(cm)
    print("\nRelatório de Classificação:")
    print(classification_report(y_val, y_pred))

    return acuracia, f1, cm


diretorios_plasticoscm = ['ABS', 'PA', 'PC', 'PE', 'PES', 'PET', 'PMMA', 'PP', 'PS', 'PTFE', 'PU', 'PVC']
# Chamar a função de validação simples e armazenar os resultados
acuracia, f1, cm = validacao_simples(best_modelo, train_data_pca, train_labels, test_data_pca, test_labels, oversampling=True, diretorios_plasticos=diretorios_plasticoscm)

# Verificar a forma da matriz de confusão
print(f"Forma da matriz de confusão: {cm.shape}")

# Exibir matriz de confusão
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=diretorios_plasticoscm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Matriz de Confusão')
plt.xlabel('Classe Prevista')
plt.ylabel('Classe Verdadeira')
plt.xticks(rotation=90)  # Girar rótulos do eixo x para a vertical
plt.show()  # Garante que a matriz de confusão será exibida

In [ ]:
#Curva de aprendizado para detectar overfitting

# Definição da função de acurácia
scoring = make_scorer(accuracy_score)

# Cálculo da curva de aprendizado
train_sizes, train_scores, test_scores = learning_curve(
    best_modelo, train_data_pca, train_labels, cv=None, n_jobs=None, train_sizes=np.linspace(0.1, 1.0, 10), scoring=scoring
)

# Calculando a média e o desvio padrão das pontuações de treinamento e teste
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Plotagem
plt.figure()
plt.title("Curva de Aprendizado para o KNN")
plt.xlabel("Tamanho do Conjunto de Treinamento")
plt.ylabel("Acurácia")
plt.grid()

# Preenchendo a área ao redor da linha para erro padrão
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.1, color="g")

# Plotando as linhas médias
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Treinamento")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Teste")
plt.ylim(0,1.2)
plt.legend(loc="best")
plt.show()
